# Implemented Subgroup Disovery Algorithms
For this example is used the UCI adult dataset where the objective is to predict whether a person makes more (label 1) or less (0) than $50,000 a year.

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.tree import DecisionTreeClassifier
#Import dataset
d = fetch_openml(data_id=1590, as_frame=True)
X = d.data
d_train=pd.get_dummies(X)
y_true = (d.target == '>50K') * 1
#training the classifier
classifier = DecisionTreeClassifier(min_samples_leaf=10, max_depth=4)
classifier.fit(d_train, y_true)
#Producing y_pred
y_pred = classifier.predict(d_train)

This package offers two Top-K Subgroup Discovery Algorithms: **BeamSearch** and **DSSD**.

## BeamSearch Algorithm
BeamSEarch is an euristic algorithm representing a good between the completeness of exhaustive search algorithms and the speed of greedy algorithms. <br/> This trade-off can be adjusted via the beam width. For a beam width that tends to infinity the algorithm becomes an exhaustive search algoritm, instead with a beam width equal to 1 the algorithm becomes a greedy one.

In [2]:
# Beam search algorithm usage
from fairsd import SubgroupDiscoveryTask
from fairsd import BeamSearch
task = SubgroupDiscoveryTask(X, y_true, y_pred)
resultset = BeamSearch().execute(task)

The beam width is 20 by default but it is possible specify a different value:

In [3]:
resultset = BeamSearch(beam_width=10).execute(task)
print(resultset)

education = "Bachelors" AND marital-status = "Married-civ-spouse" 
education = "Bachelors" AND marital-status = "Married-civ-spouse" AND capital-loss = (-infinite, 0.0] 
education = "Bachelors" AND marital-status = "Married-civ-spouse" AND capital-gain = (-infinite, 0.0] 
education = "Bachelors" AND marital-status = "Married-civ-spouse" AND race = "White" 
education = "Bachelors" AND marital-status = "Married-civ-spouse" AND sex = "Male" 



### Redundancy
As we can see, the returned result set is somehow redundant: many variants of, essentially, the same discriminated subgroup are presents in it. <br/>
Many top-k sg-discovery algorithms suffers of this problem.<br/>
To solve this problem it is suggested to use the DSSD algorithm.

## DSSD (Diverse Subgroup Set Discovery) algorithm
 This algorithm is a variant of the Beam Search Algorithm that also take into account the redundancy of the generated subgroups.<br/>
In this package a cover-based redundancy definition is used: roughly, the more tuples two subgroups have in common, the more they are considered redundant. <br/>
The degree to which redundancy is mitigated is determined by the alpha parameter. the more a is high, the less the subgroups redundancy is taken into account. Alpha must be between zero and 1, and, the more alpha is high, the less the subgroups redundancy is taken into account.<br>

This algorithm is described in details in the Van Leeuwen and Knobbe's paper "Diverse Subgroup Set Discovery".


In [4]:
# DSSD algorithm usage
from fairsd import DSSD
resultset = DSSD(beam_width=10, a = 0.9).execute(task) # "a" parameter represents alpha, 0.9 by default 
print(resultset)

education = "Bachelors" AND marital-status = "Married-civ-spouse" 
education = "Bachelors" AND relationship = "Husband" 
education-num = (13.0, +infinite] AND marital-status = "Married-civ-spouse" 
education-num = (13.0, +infinite] AND race = "Asian-Pac-Islander" 
relationship = "Husband" AND capital-gain = (7298.0, +infinite] 



As we can see now, just by looking at the descriptions in the result set, the redundancy sems very attenuated.